In [ ]:
%%capture
!pip install llama-index==0.10.25 llama-index-embeddings-fastembed qdrant-client llama-index-vector-stores-qdrant llama-index-llms-cohere

In [ ]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv("../.env")

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

In [ ]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [ ]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [ ]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [ ]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model, setup_vector_store

COLLECTION_NAME = "words-of-the-senpai-naive"

setup_llm(api_key=CO_API_KEY)

setup_embed_model()

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

# Load Docstore

In [ ]:
from utils import get_documents_from_docstore

documents = get_documents_from_docstore("../data/words-of-the-senpais")

# Ingest to VectorDatabase

In [ ]:
from llama_index.core.constants import DEFAULT_CHUNK_SIZE
from llama_index.core.node_parser.text import SentenceSplitter

from utils import ingest

tranforms = [
    SentenceSplitter(chunk_size=DEFAULT_CHUNK_SIZE), 
    Settings.embed_model
    ]

ingest(
    documents=documents,
    transformations=tranforms,
    vector_store=vector_store
)

In [ ]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
    )

# Build index over vector database


In [ ]:
from utils import create_index, create_query_engine

index = create_index(vector_store=vector_store, storage_context=storage_context)

query_engine = create_query_engine(index=index, mode="query")

# Query Pipelines

In [ ]:
from utils import create_query_pipeline

chain = [Settings.llm, query_engine]

query_pipeline = create_query_pipeline(chain)

# Query

In [ ]:
query_pipeline.run("What mental models, principles or philosophies are most valuable for making important life decisions and navigating an uncertain world?")

In [ ]:
query_pipeline.run("What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?")

In [ ]:
query_pipeline.run("What is the essence of a life well-lived, and what are the key components of a good life that everyone should strive for?")

In [ ]:
query_pipeline.run("What role does luck play in achieving outsized success and wealth creation? Is capitalizing on luck a skill that can be cultivated?")

In [ ]:
query_pipeline.run("What are some commonalities between Naval Ravikant and Nassim Taleb's thinking on luck?")